# Import Dependencies

In [1]:
import pandas as pd
from random import randint

# Read in the Data

The only change that should be needed is the filepath. 
The rest of the notebook should run without any interaction needed.

In [2]:
#file_name = '../EIL51.xlsx'
file_name = '../Lab Data.xlsx'
distances = pd.read_excel(file_name, index_col=None)

TOUR_LENGTH = len(distances)

# Greedy Randomized Heuristic

In [3]:
def valid(current_tour):
    # Drop cities that are already in the current tour
    valid_nbrs = [element for element in range(TOUR_LENGTH) if element not in current_tour]
    return valid_nbrs


def nearestish_neighbor(starting_city=None, tour=None, top_l=3):   
    '''
        Optionally takes in a starting city, tour, and/or alpha value.
        If no starting city is provided a random one is chosen
        If no current tour is provided it is instantiated as an empty list
        The top_l is the number of in the restricted candidates list

        Returns a tour and the corresponding length
    '''
    tour = tour or []
    current_city = starting_city if isinstance(starting_city, int) else randint(0,TOUR_LENGTH-1)
    tour_length = 0
    tour.append(current_city)   # add starting city to tour    
    
    for _ in range(TOUR_LENGTH-1):
        next_city, distance_to_next = nearestish(current_city, tour, top_l)
        tour.append(next_city)
        current_city = next_city
        tour_length += distance_to_next

    tour.append(tour[0]) # The TSP requires returning home at the end.
    tour_length += distances[tour[-2]][tour[-1]] # Add the return distance
    
    return tour, tour_length


def nearestish(current_city, current_tour, top_l=3):
    '''
        Given current city and list of current tour find a nearish neighbor
        from within the l shortest options.

        Returns index of next city and distance to that city
    '''
    # Get distances from current city
    # Filter for valid destinations
    candidates = distances[current_city].filter(valid(current_tour))

    # Select l best options (RCL)
    rcl = candidates.nsmallest(top_l)

    # Randomly Select from RCL
    nearish_city = rcl.sample(1)

    # Return Destination and distance to it
    return nearish_city.index[0], nearish_city.values[0]

# Local Search

In [4]:
def cost(tour):
    cost = 0
    for i in range(0, TOUR_LENGTH): 
        cost += distances.iat[tour[i],tour[i+1]]
    return cost

def two_opt(tour):
    '''
        Takes a feasible tour and iteratively swaps destinations of two cities;
        effectively reversing the order of the tour between the two.
        The function returns the tour with the lowest distance cost.
    '''
    tour_len = len(tour)
    best_tour = tour[:]
    best_cost = cost(best_tour)
    improved = True

    while improved:
        improved = False
        for i in range(1, tour_len-3):          #i is index of first city
            for j in range(i+2, tour_len):      #j is index of second city
                new_tour = best_tour[:]               #clone tour
                # swap the order of the tour between these two cities
                new_tour[i:j] = best_tour[j-1:i-1:-1] 
                new_cost = cost(new_tour)
                if new_cost < best_cost:
                    best_tour = new_tour[:]
                    best_cost = new_cost
                    improved = True
        tour = best_tour[:]
        #print("I found a new tour:",best_tour, best_cost)

    return best_tour, best_cost

# GRASP

Bringin' it all together.

In [5]:
def graspy(iterations=3, top_l=3):
    '''
        Implements a GRASP metaheuristic using nearest neighbor as the base 
        greedy heuristic and 2-opt as the local search heuristic. 

        Optionally takes a number of iterations to repeat the initial 
        randomized greedy and local searches, and a top_l corresponding to the 
        number of candidates in the semi-greedy constructive heuristic.
    '''
    best_tour = []
    best_cost = 100**10 # Large cost to guarantee replacement on iter 1
    for _ in range(iterations):
        # Step 1: Greedy Random, nearish neighbor
        tour, _ = nearestish_neighbor(top_l=top_l)
        # Step 2: Adaptive Search, local search
        improved_tour, improved_cost = two_opt(tour)
        # Step 3: Procedure, Keep the best one
        if improved_cost < best_cost:
            best_tour = improved_tour
            best_cost = improved_cost

    return best_tour, best_cost

## Testing the Function

In [22]:
t, c = graspy(iterations=10)
print(f'Tour: {t}\nCost: {c}')

Tour: [2, 5, 6, 7, 10, 9, 13, 1, 8, 4, 11, 3, 14, 0, 12, 2]
Cost: 217
